# 0616098 黃秉茂 ML HW4 SVM and ANN

### import module

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

# Data Input

## read json

In [2]:
dataset = pd.read_json('train.json')

In [3]:
print(dataset)

          id      cuisine                                        ingredients
0      22675      italian  [1% low-fat cottage cheese, low-fat marinara s...
1      32288  southern_us  [brown sugar, salt, eggs, butter, chopped peca...
2      44406         thai  [red chili peppers, bell pepper, garlic, fish ...
3      29355     moroccan  [water, green tea leaves, tangerine, fresh min...
4      39350      chinese  [vegetable oil, chile sauce, tomato paste, gar...
...      ...          ...                                                ...
29769   2278     japanese  [soy sauce, sesame oil, garlic, sake, flour, g...
29770    474   vietnamese  [mint, garlic sauce, chinese chives, rice nood...
29771  44229       indian  [potatoes, vegetable broth, oil, cashew nuts, ...
29772  20311  southern_us  [butter, powdered sugar, cream cheese, soften,...
29773  32823      chinese  [savoy cabbage, dumpling wrappers, ginger, soy...

[29774 rows x 3 columns]


# Data Preprocessing

In [4]:
from sklearn.utils import shuffle
from sklearn import preprocessing

## drop id

In [5]:
dataset = dataset.drop(columns=dataset.columns[0])

In [6]:
dataset_last_word = dataset.copy()
dataset_last_2_word = dataset.copy()

## drop redundancy and limit the number of words in a string which is in the list in a column using DataFrame

In [7]:
word_limit = 1
dataset_last_word[dataset_last_word.columns[-1]] = dataset_last_word[dataset_last_word.columns[-1]].apply(lambda x : [' '.join(((item.split('(')[0]).split(',')[0]).split()[-word_limit:]) for item in x])
word_limit = 2
dataset_last_2_word[dataset_last_2_word.columns[-1]] = dataset_last_2_word[dataset_last_2_word.columns[-1]].apply(lambda x : [' '.join(((item.split('(')[0]).split(',')[0]).split()[-word_limit:]) for item in x])

## Shuffle the data

In [8]:
dataset_shuffle = shuffle(dataset)
dataset_last_word_shuffle = shuffle(dataset_last_word)
dataset_last_2_word_shuffle = shuffle(dataset_last_2_word)

In [9]:
x = dataset_shuffle[dataset_shuffle.columns[-1]]
x_last_word = dataset_last_word_shuffle[dataset_last_word_shuffle.columns[-1]]
x_last_2_word = dataset_last_2_word_shuffle[dataset_last_2_word_shuffle.columns[-1]]
y = dataset_shuffle[dataset_shuffle.columns[0]]
y_last_word = dataset_last_word_shuffle[dataset_last_word_shuffle.columns[0]]
y_last_2_word = dataset_last_2_word_shuffle[dataset_last_2_word_shuffle.columns[0]]

## Transform data format and shape so your model can process them

### one hot encode a series of lists in pandas

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
# mlb = MultiLabelBinarizer()
# X = pd.DataFrame(mlb.fit_transform(x), columns=mlb.classes_, index=x.index)
# X_last_word = pd.DataFrame(mlb.fit_transform(x_last_word), columns=mlb.classes_, index=x_last_word.index)
# X_last_2_word = pd.DataFrame(mlb.fit_transform(x_last_2_word), columns=mlb.classes_, index=x_last_2_word.index)

mlb = MultiLabelBinarizer()
mlb.fit(x)
mlb_last_word = MultiLabelBinarizer()
mlb_last_word.fit(x_last_word)
mlb_last_2_word = MultiLabelBinarizer()
mlb_last_2_word.fit(x_last_2_word)
X = pd.DataFrame(mlb.transform(x), columns=mlb.classes_, index=x.index)
X_last_word = pd.DataFrame(mlb_last_word.transform(x_last_word), columns=mlb_last_word.classes_, index=x_last_word.index)
X_last_2_word = pd.DataFrame(mlb_last_2_word.transform(x_last_2_word), columns=mlb_last_2_word.classes_, index=x_last_2_word.index)

In [12]:
X = np.array(X)
X = X.astype(float)
X_last_word = np.array(X_last_word)
X_last_word = X_last_word.astype(float)
X_last_2_word = np.array(X_last_2_word)
X_last_2_word = X_last_2_word.astype(float)
Y = np.array(y)
Y_last_word = np.array(y_last_word)
Y_last_2_word = np.array(y_last_2_word)

In [13]:
print('no word limit', X.shape)
print('last word', X_last_word.shape)
print('last 2 word', X_last_2_word.shape)

no word limit (29774, 6231)
last word (29774, 1749)
last 2 word (29774, 4897)


## Any data augmentation that can boost your final results

### use PCA to get important features and alleviate amount of calculating

In [14]:
from sklearn.decomposition import PCA

In [15]:
def pca_with_proper_n_com(data, n_com_start):
    n_com = n_com_start
    explained_ratio = 0
    explained_ratio_threshold = 80
    while explained_ratio < explained_ratio_threshold:
        n_com += 1
        pca = PCA(n_components=n_com)
        # calculate variance ratios
        pca.fit(data)
        cumulative_sum_of_variance_explained = np.cumsum(pca.explained_variance_ratio_ * 100)
        explained_ratio = cumulative_sum_of_variance_explained[-1]
    pca = PCA(n_components = n_com)
    data_pca = pca.fit_transform(data)
    return data_pca, n_com, explained_ratio

In [16]:
X_pca, X_proper_n_com, X_explained_ratio = pca_with_proper_n_com(X, 500)
X_pca_last_word, X_proper_n_com_last_word, X_explained_ratio_last_word = pca_with_proper_n_com(X_last_word, 105)
X_pca_last_2_word, X_proper_n_com_last_2_word, X_explained_ratio_last_2_word = pca_with_proper_n_com(X_last_2_word, 380)
print('n_com:', X_proper_n_com, 'explained variance ratio:', X_explained_ratio)
print('n_com:', X_proper_n_com_last_word, 'explained variance ratio:', X_explained_ratio_last_word)
print('n_com:', X_proper_n_com_last_2_word, 'explained variance ratio:', X_explained_ratio_last_2_word)

n_com: 508 explained variance ratio: 80.02014647908625
n_com: 109 explained variance ratio: 80.00176176473056
n_com: 384 explained variance ratio: 80.03133260816708


In [17]:
print('no word limit', X_pca.shape)
print('last word', X_pca_last_word.shape)
print('last 2 word', X_pca_last_2_word.shape)

no word limit (29774, 508)
last word (29774, 109)
last 2 word (29774, 384)


## label encorder for one hot encoder

In [18]:
from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# Y_le = labelencoder.fit_transform(Y)
# Y_le_last_word = labelencoder.fit_transform(Y_last_word)
# Y_le_last_2_word = labelencoder.fit_transform(Y_last_2_word)

label_encoder = LabelEncoder()
Y_le_model = label_encoder.fit(Y)
Y_le = Y_le_model.transform(Y)
label_encoder_last_word = LabelEncoder()
Y_le_model_last_word = label_encoder_last_word.fit(Y_last_word)
Y_le_last_word = Y_le_model_last_word.transform(Y_last_word)
label_encoder_last_2_word = LabelEncoder()
Y_le_model_last_2_word = label_encoder_last_2_word.fit(Y_last_2_word)
Y_le_last_2_word = Y_le_model_last_2_word.transform(Y_last_2_word)

## one hot encode label for ANN

In [19]:
from tensorflow.keras.utils import to_categorical

In [20]:
Y_onehot = to_categorical(Y_le)
Y_onehot_last_word = to_categorical(Y_le_last_word)
Y_onehot_last_2_word = to_categorical(Y_le_last_2_word)

# Model Construction

## Support Vector Machine

### import module

In [21]:
from sklearn.svm import LinearSVC

In [22]:
from sklearn.svm import SVC

In [23]:
linear_svc_model = LinearSVC()

In [24]:
svc_linear_model = SVC(kernel = 'linear')
svc_poly_model = SVC(kernel = 'poly')
svc_rbf_model = SVC(kernel = 'rbf')

## Artificial Neural Networks

### import module

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import clone_model

In [26]:
model = Sequential()
model.add(Dense(units=3000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=200, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=len(set(Y)), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_last_word = clone_model(model)
model_last_word.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_last_2_word = clone_model(model)
model_last_2_word.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


- I use tensorflow as ANN framework
* 用relu和selu和linear是因為我認為這是線性關係
* 用softmax當作多類別機率
* Dropout避免overfitting
* loss: 設定 Loss Function, 因為是multiple class，這邊選定 Cross Entropy 作為 Loss Function.
* optimizer: 設定訓練時的優化方法, 在深度學習使用 adam (Adam: A Method for Stochastic Optimization) 可以更快收斂, 考慮momentum和隨次數減少learning rate以提高準確率.
* metrics: 設定評估模型的方式是 accuracy 準確率.

## Validation

### Holdout validation 

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=0)
x_train_last_word, x_val_last_word, y_train_last_word, y_val_last_word = train_test_split(X_last_word, Y_last_word, test_size=0.3, random_state=0)
x_train_last_2_word, x_val_last_2_word, y_train_last_2_word, y_val_last_2_word = train_test_split(X_last_2_word, Y_last_2_word, test_size=0.3, random_state=0)

x_train_pca, x_val_pca, y_train_pca, y_val_pca = train_test_split(X_pca, Y, test_size=0.3, random_state=0)
x_train_pca_last_word, x_val_pca_last_word, y_train_pca_last_word, y_val_pca_last_word = train_test_split(X_pca_last_word, Y_last_word, test_size=0.3, random_state=0)
x_train_pca_last_2_word, x_val_pca_last_2_word, y_train_pca_last_2_word, y_val_pca_last_2_word = train_test_split(X_pca_last_2_word, Y_last_2_word, test_size=0.3, random_state=0)

x_train_onehot, x_val_onehot, y_train_onehot, y_val_onehot = train_test_split(X, Y_onehot, test_size=0.3, random_state=0)
x_train_onehot_last_word, x_val_onehot_last_word, y_train_onehot_last_word, y_val_onehot_last_word = train_test_split(X_last_word, Y_onehot_last_word, test_size=0.3, random_state=0)
x_train_onehot_last_2_word, x_val_onehot_last_2_word, y_train_onehot_last_2_word, y_val_onehot_last_2_word = train_test_split(X_last_2_word, Y_onehot_last_2_word, test_size=0.3, random_state=0)

# Results

In [29]:
linear_svc = linear_svc_model.fit(x_train, y_train)
pred_linear_svc = linear_svc.predict(x_val)

linear_svc_last_word = linear_svc_model.fit(x_train_last_word, y_train_last_word)
pred_linear_svc_last_word = linear_svc_last_word.predict(x_val_last_word)

linear_svc_last_2_word = linear_svc_model.fit(x_train_last_2_word, y_train_last_2_word)
pred_linear_svc_last_2_word = linear_svc_last_2_word.predict(x_val_last_2_word)

In [30]:
svc_linear = svc_linear_model.fit(x_train_pca, y_train_pca)
pred_svc_linear = svc_linear.predict(x_val_pca)

svc_linear_last_word = svc_linear_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_linear_last_word = svc_linear_last_word.predict(x_val_pca_last_word)

svc_linear_last_2_word = svc_linear_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_linear_last_2_word = svc_linear_last_2_word.predict(x_val_pca_last_2_word)

In [31]:
svc_poly = svc_poly_model.fit(x_train_pca, y_train_pca)
pred_svc_poly = svc_poly.predict(x_val_pca)

svc_poly_last_word = svc_poly_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_poly_last_word = svc_poly_last_word.predict(x_val_pca_last_word)

svc_poly_last_2_word = svc_poly_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_poly_last_2_word = svc_poly.predict(x_val_pca_last_2_word)

In [32]:
svc_rbf = svc_rbf_model.fit(x_train_pca, y_train_pca)
pred_svc_rbf = svc_rbf.predict(x_val_pca)

svc_rbf_last_word = svc_rbf_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_rbf_last_word = svc_rbf.predict(x_val_pca_last_word)

svc_rbf_last_2_word = svc_rbf_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_rbf_last_2_word = svc_rbf_last_2_word.predict(x_val_pca_last_2_word)

In [33]:
print('no word limit')
train_model = model.fit(x=x_train_onehot, y=y_train_onehot, epochs=8, batch_size=1024, verbose=2)
pred_ANN = model.predict_classes(x_val_onehot)
print('last word')
train_model_last_word = model_last_word.fit(x=x_train_onehot_last_word, y=y_train_onehot_last_word, epochs=8, batch_size=1024, verbose=2)
pred_ANN_last_word = model_last_word.predict_classes(x_val_onehot_last_word)
print('last 2 word')
train_model_last_2_word = model_last_2_word.fit(x=x_train_onehot_last_2_word, y=y_train_onehot_last_2_word, epochs=8, batch_size=1024, verbose=2)
pred_ANN_last_2_word = model_last_2_word.predict_classes(x_val_onehot_last_2_word)

no word limit
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/8
20841/20841 - 1s - loss: 1.9363 - acc: 0.4826
Epoch 2/8
20841/20841 - 1s - loss: 0.8872 - acc: 0.7543
Epoch 3/8
20841/20841 - 1s - loss: 0.5140 - acc: 0.8566
Epoch 4/8
20841/20841 - 1s - loss: 0.3147 - acc: 0.9123
Epoch 5/8
20841/20841 - 1s - loss: 0.1967 - acc: 0.9504
Epoch 6/8
20841/20841 - 1s - loss: 0.1231 - acc: 0.9714
Epoch 7/8
20841/20841 - 1s - loss: 0.0778 - acc: 0.9844
Epoch 8/8
20841/20841 - 1s - loss: 0.0514 - acc: 0.9916
last word
Epoch 1/8
20841/20841 - 0s - loss: 1.9636 - acc: 0.4535
Epoch 2/8
20841/20841 - 0s - loss: 1.1143 - acc: 0.6761
Epoch 3/8
20841/20841 - 0s - loss: 0.8576 - acc: 0.7493
Epoch 4/8
20841/20841 - 0s - loss: 0.7138 - acc: 0.7882
Epoch 5/8
20841/20841 - 0s - loss: 0.6114 - acc: 0.8200
Epoch 6/8
20841/20841 - 0s - loss: 0.5157 - acc: 0.8492
Epoch 7/8
20841/20841 - 0s - loss: 0.4352 - acc: 0.8725
Epoch 8/8
20841/20841 - 0s - loss: 0.3645 

In [34]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [35]:
print('no word limit')
print('\tConfusion matrix')
print('\t\tlinear_svc:', confusion_matrix(y_val, pred_linear_svc))
print('\t\tsvc_linear:', confusion_matrix(y_val_pca, pred_svc_linear))
print('\t\tsvc_poly:', confusion_matrix(y_val_pca, pred_svc_poly))
print('\t\tsvc_rbf:', confusion_matrix(y_val_pca, pred_svc_rbf))
print('\t\tANN:', confusion_matrix(np.argmax(y_val_onehot, axis=1), pred_ANN))
print('\tAccuracy')
print('\t\tlinear_svc:', accuracy_score(y_val, pred_linear_svc))
print('\t\tsvc_linear:', accuracy_score(y_val_pca, pred_svc_linear))
print('\t\tsvc_poly:', accuracy_score(y_val_pca, pred_svc_poly))
print('\t\tsvc_rbf:', accuracy_score(y_val_pca, pred_svc_rbf))
print('\t\tANN:', accuracy_score(np.argmax(y_val_onehot, axis=1), pred_ANN))
print('\tSensitivity(Recall)')
print('\t\tlinear_svc:', recall_score(y_val, pred_linear_svc, average='weighted'))
print('\t\tsvc_linear:', recall_score(y_val_pca, pred_svc_linear, average='weighted'))
print('\t\tsvc_poly:', recall_score(y_val_pca, pred_svc_poly, average='weighted'))
print('\t\tsvc_rbf:', recall_score(y_val_pca, pred_svc_rbf, average='weighted'))
print('\t\tANN:', recall_score(np.argmax(y_val_onehot, axis=1), pred_ANN, average='weighted'))
print('\tPrecision')
print('\t\tlinear_svc:', precision_score(y_val, pred_linear_svc, average='weighted'))
print('\t\tsvc_linear:', precision_score(y_val_pca, pred_svc_linear, average='weighted'))
print('\t\tsvc_poly:', precision_score(y_val_pca, pred_svc_poly, average='weighted'))
print('\t\tsvc_rbf:', precision_score(y_val_pca, pred_svc_rbf, average='weighted'))
print('\t\tANN:', precision_score(np.argmax(y_val_onehot, axis=1), pred_ANN, average='weighted'))

print('last word')
print('\tConfusion matrix')
print('\t\tlinear_svc:', confusion_matrix(y_val_last_word, pred_linear_svc_last_word))
print('\t\tsvc_linear:', confusion_matrix(y_val_pca_last_word, pred_svc_linear_last_word))
print('\t\tsvc_poly:', confusion_matrix(y_val_pca_last_word, pred_svc_poly_last_word))
print('\t\tsvc_rbf:', confusion_matrix(y_val_pca_last_word, pred_svc_rbf_last_word))
print('\t\tANN:', confusion_matrix(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word))
print('\tAccuracy')
print('\t\tlinear_svc:', accuracy_score(y_val_last_word, pred_linear_svc_last_word))
print('\t\tsvc_linear:', accuracy_score(y_val_pca_last_word, pred_svc_linear_last_word))
print('\t\tsvc_poly:', accuracy_score(y_val_pca_last_word, pred_svc_poly_last_word))
print('\t\tsvc_rbf:', accuracy_score(y_val_pca_last_word, pred_svc_rbf_last_word))
print('\t\tANN:', accuracy_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word))
print('\tSensitivity(Recall)')
print('\t\tlinear_svc:', recall_score(y_val_last_word, pred_linear_svc_last_word, average='weighted'))
print('\t\tsvc_linear:', recall_score(y_val_pca_last_word, pred_svc_linear_last_word, average='weighted'))
print('\t\tsvc_poly:', recall_score(y_val_pca_last_word, pred_svc_poly_last_word, average='weighted'))
print('\t\tsvc_rbf:', recall_score(y_val_pca_last_word, pred_svc_rbf_last_word, average='weighted'))
print('\t\tANN:', recall_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word, average='weighted'))
print('\tPrecision')
print('\t\tlinear_svc:', precision_score(y_val_last_word, pred_linear_svc_last_word, average='weighted'))
print('\t\tsvc_linear:', precision_score(y_val_pca_last_word, pred_svc_linear_last_word, average='weighted'))
print('\t\tsvc_poly:', precision_score(y_val_pca_last_word, pred_svc_poly_last_word, average='weighted'))
print('\t\tsvc_rbf:', precision_score(y_val_pca_last_word, pred_svc_rbf_last_word, average='weighted'))
print('\t\tANN:', precision_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word, average='weighted'))

print('last 2 word')
print('\tConfusion matrix')
print('\t\tlinear_svc:', confusion_matrix(y_val_last_2_word, pred_linear_svc_last_2_word))
print('\t\tsvc_linear:', confusion_matrix(y_val_pca_last_2_word, pred_svc_linear_last_2_word))
print('\t\tsvc_poly:', confusion_matrix(y_val_pca_last_2_word, pred_svc_poly_last_2_word))
print('\t\tsvc_rbf:', confusion_matrix(y_val_pca_last_2_word, pred_svc_rbf_last_2_word))
print('\t\tANN:', confusion_matrix(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word))
print('\tAccuracy')
print('\t\tlinear_svc:', accuracy_score(y_val_last_2_word, pred_linear_svc_last_2_word))
print('\t\tsvc_linear:', accuracy_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word))
print('\t\tsvc_poly:', accuracy_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word))
print('\t\tsvc_rbf:', accuracy_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word))
print('\t\tANN:', accuracy_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word))
print('\tSensitivity(Recall)')
print('\t\tlinear_svc:', recall_score(y_val_last_2_word, pred_linear_svc_last_2_word, average='weighted'))
print('\t\tsvc_linear:', recall_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word, average='weighted'))
print('\t\tsvc_poly:', recall_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word, average='weighted'))
print('\t\tsvc_rbf:', recall_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word, average='weighted'))
print('\t\tANN:', recall_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word, average='weighted'))
print('\tPrecision')
print('\t\tlinear_svc:', precision_score(y_val_last_2_word, pred_linear_svc_last_2_word, average='weighted'))
print('\t\tsvc_linear:', precision_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word, average='weighted'))
print('\t\tsvc_poly:', precision_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word, average='weighted'))
print('\t\tsvc_rbf:', precision_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word, average='weighted'))
print('\t\tANN:', precision_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word, average='weighted'))

no word limit
	Confusion matrix
		linear_svc: [[  55    1    4    0    5    0    0    4    1    5    0    1    0   10
     0    0    6    6    4    0]
 [   0   74    2    7    1   19    1    4   10   10    2    1    1    5
     1    5   22    2    0    4]
 [   2    1  257    1    0   16    1    4    0   19    1    0    1    8
     0    3   44    7    1    1]
 [   1    0    5  466   14    3    2    4    0    8    1   17   17    5
     0    4   12    2   12    8]
 [   1    0    1   21  103    5    0    5    1   10    3    2    1    8
     0    2   12    1    8    5]
 [   6   12   10    4    3  327    5    4   11  118    1    2    0    6
     6    3   43   21    1    4]
 [   0    3    0    1    2    6  188    9    1   38    0    0    0    2
     3    1    8    4    0    0]
 [   0    0    0    4    3   11    3  577    3   10    2    6    1    8
    21    0   10    1    7    3]
 [   1   25    3    1    0   12    1    5   67   15    2    1    0    3
     3    4   23    1    0    0]
 [   2   

		svc_poly: 0.5775215493115414
		svc_rbf: 0.7171163103100862
		ANN: 0.7649166013657226
	Precision
		linear_svc: 0.7482292692149584
		svc_linear: 0.7100263550577157
		svc_poly: 0.6861533023185841
		svc_rbf: 0.7274333486501594
		ANN: 0.7627224902561239
last word
	Confusion matrix
		linear_svc: [[  36    1    6    2    2    3    1    3    1   14    3    1    1   15
     0    1    5    1    6    0]
 [   0   60    2    4    2   25    1    2   10   19    0    1    0    2
     0    4   30    2    1    0]
 [   2    2  218    1    1   15    4    2    2   43    1    0    0   10
     1    4   40    3    0    0]
 [   0    2    1  467    7    6    0    7    0   22    2   13   15   11
     2    1    8    2   13    8]
 [   1    0    3   17   71    9    0    6    0   10    3    4    3    8
     0    1   11    2   11    4]
 [   3   14    4    7    1  329    9    3    7  140    4    2    2   10
     3    7   55    8    3    2]
 [   0    1    3    1    0    5  155    5    0   61    1    0    0    5
     

		svc_poly: 0.6497257360349267
		svc_rbf: 0.6845404679279078
		ANN: 0.7411843725512146
	Precision
		linear_svc: 0.7131890614718477
		svc_linear: 0.6540039402529008
		svc_poly: 0.6872679527735676
		svc_rbf: 0.6972520042246662
		ANN: 0.7403820188931293
last 2 word
	Confusion matrix
		linear_svc: [[  57    1    1    1    1    3    0    2    2    1    3    0    0   13
     0    1    5    2    6    0]
 [   2   87    1    0    0   26    3    7   20   15    1    0    0    4
     0    2   32    1    0    0]
 [   3    5  245    0    0    8    2    4    0   22    2    0    0    9
     1    2   60    4    0    0]
 [   1    0    2  457    8    4    1    5    1   13    1   14   16    9
     2    2   13    1   11   15]
 [   6    0    0   16  107    4    0    4    0    6    2    3    2    8
     0    1   11    3    4    1]
 [   1   10    4    4    3  356    8    7   12  113    5    1    0   11
     5    8   31   19    0    1]
 [   1    1    1    1    0   12  178    7    0   37    0    1    2    1
   

		svc_rbf: 0.7176760326877869
		ANN: 0.768722713534087
	Precision
		linear_svc: 0.7508727612727286
		svc_linear: 0.7084583473370906
		svc_poly: 0.6938962094997846
		svc_rbf: 0.7222459572669201
		ANN: 0.768157488243125


In [49]:
print('Accuracy')
print('\tno word limit', 'last word', 'last 2 word', sep='\t\t\t\t')
print('\tlinear_svc:', accuracy_score(y_val, pred_linear_svc),'\t\t', accuracy_score(y_val_last_word, pred_linear_svc_last_word), '\t\t', accuracy_score(y_val_last_2_word, pred_linear_svc_last_2_word))
print('\tsvc_linear:', accuracy_score(y_val_pca, pred_svc_linear),'\t\t', accuracy_score(y_val_pca_last_word, pred_svc_linear_last_word), '\t\t', accuracy_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word))
print('\tsvc_poly:', accuracy_score(y_val_pca, pred_svc_poly),'\t\t', accuracy_score(y_val_pca_last_word, pred_svc_poly_last_word),'\t\t', accuracy_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word))
print('\tsvc_rbf:', accuracy_score(y_val_pca, pred_svc_rbf),'\t\t', accuracy_score(y_val_pca_last_word, pred_svc_rbf_last_word),'\t\t', accuracy_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word))
print('\tANN:', accuracy_score(np.argmax(y_val_onehot, axis=1), pred_ANN),'\t\t', accuracy_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word),'\t\t', accuracy_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word))

print('Sensitivity(Recall)')
print('\tno word limit', 'last word', 'last 2 word', sep='\t\t\t\t')
print('\tlinear_svc:', recall_score(y_val, pred_linear_svc, average='weighted'),'\t\t', recall_score(y_val_last_word, pred_linear_svc_last_word, average='weighted'), '\t\t', recall_score(y_val_last_2_word, pred_linear_svc_last_2_word, average='weighted'))
print('\tsvc_linear:', recall_score(y_val_pca, pred_svc_linear, average='weighted'),'\t\t', recall_score(y_val_pca_last_word, pred_svc_linear_last_word, average='weighted'), '\t\t', recall_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word, average='weighted'))
print('\tsvc_poly:', recall_score(y_val_pca, pred_svc_poly, average='weighted'),'\t\t', recall_score(y_val_pca_last_word, pred_svc_poly_last_word, average='weighted'),'\t\t', recall_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word, average='weighted'))
print('\tsvc_rbf:', recall_score(y_val_pca, pred_svc_rbf, average='weighted'),'\t\t', recall_score(y_val_pca_last_word, pred_svc_rbf_last_word, average='weighted'),'\t\t', recall_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word, average='weighted'))
print('\tANN:', recall_score(np.argmax(y_val_onehot, axis=1), pred_ANN, average='weighted'),'\t\t', recall_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word, average='weighted'),'\t\t', recall_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word, average='weighted'))

print('Precision')
print('\tno word limit', 'last word', 'last 2 word', sep='\t\t\t\t')
print('\tlinear_svc:', precision_score(y_val, pred_linear_svc, average='weighted'),'\t\t', precision_score(y_val_last_word, pred_linear_svc_last_word, average='weighted'), '\t\t', precision_score(y_val_last_2_word, pred_linear_svc_last_2_word, average='weighted'))
print('\tsvc_linear:', precision_score(y_val_pca, pred_svc_linear, average='weighted'),'\t\t', precision_score(y_val_pca_last_word, pred_svc_linear_last_word, average='weighted'), '\t\t', precision_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word, average='weighted'))
print('\tsvc_poly:', precision_score(y_val_pca, pred_svc_poly, average='weighted'),'\t\t', precision_score(y_val_pca_last_word, pred_svc_poly_last_word, average='weighted'),'\t\t', precision_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word, average='weighted'))
print('\tsvc_rbf:', precision_score(y_val_pca, pred_svc_rbf, average='weighted'),'\t\t', precision_score(y_val_pca_last_word, pred_svc_rbf_last_word, average='weighted'),'\t\t', precision_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word, average='weighted'))
print('\tANN:', precision_score(np.argmax(y_val_onehot, axis=1), pred_ANN, average='weighted'),'\t\t', precision_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word, average='weighted'),'\t\t', precision_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word, average='weighted'))

Accuracy
	no word limit				last word				last 2 word
	linear_svc: 0.7518190977275272 		 0.7186835329676481 		 0.7547296540915706
	svc_linear: 0.7139818649949625 		 0.6561065711407142 		 0.713534087092802
	svc_poly: 0.5775215493115414 		 0.6497257360349267 		 0.6122243367289825
	svc_rbf: 0.7171163103100862 		 0.6845404679279078 		 0.7176760326877869
	ANN: 0.7649166013657226 		 0.7411843725512146 		 0.768722713534087
Sensitivity(Recall)
	no word limit				last word				last 2 word
	linear_svc: 0.7518190977275272 		 0.7186835329676481 		 0.7547296540915706
	svc_linear: 0.7139818649949625 		 0.6561065711407142 		 0.713534087092802
	svc_poly: 0.5775215493115414 		 0.6497257360349267 		 0.6122243367289825
	svc_rbf: 0.7171163103100862 		 0.6845404679279078 		 0.7176760326877869
	ANN: 0.7649166013657226 		 0.7411843725512146 		 0.768722713534087
Precision
	no word limit				last word				last 2 word
	linear_svc: 0.7482292692149584 		 0.7131890614718477 		 0.7508727612727286
	svc_linear: 0.710026355

# Comparison & Conclusion

- ANN > LinearSVC > SVC(kernel = 'poly') > SVC(kernel = 'rbf') > SVC(kernel = 'poly')
- no word limit	> last 2 word > last word

- 普遍上Cross-Validation的表現比Holdout Validation好

In [37]:
model_name_list = ['linear_svc', 'svc_linear', 'svc_poly', 'svc_rbf', 'ANN']
data_word_limit_name_list = ['no word limit', 'last word', 'last 2 word']
accuracy_list = []

accuracy_list.append(accuracy_score(y_val, pred_linear_svc))
accuracy_list.append(accuracy_score(y_val_pca, pred_svc_linear))
accuracy_list.append(accuracy_score(y_val_pca, pred_svc_poly))
accuracy_list.append(accuracy_score(y_val_pca, pred_svc_rbf))
accuracy_list.append(accuracy_score(np.argmax(y_val_onehot, axis=1), pred_ANN))

accuracy_list.append(accuracy_score(y_val_last_word, pred_linear_svc_last_word))
accuracy_list.append(accuracy_score(y_val_pca_last_word, pred_svc_linear_last_word))
accuracy_list.append(accuracy_score(y_val_pca_last_word, pred_svc_poly_last_word))
accuracy_list.append(accuracy_score(y_val_pca_last_word, pred_svc_rbf_last_word))
accuracy_list.append(accuracy_score(np.argmax(y_val_onehot_last_word, axis=1), pred_ANN_last_word))

accuracy_list.append(accuracy_score(y_val_last_2_word, pred_linear_svc_last_2_word))
accuracy_list.append(accuracy_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word))
accuracy_list.append(accuracy_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word))
accuracy_list.append(accuracy_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word))
accuracy_list.append(accuracy_score(np.argmax(y_val_onehot_last_2_word, axis=1), pred_ANN_last_2_word))

best_accuracy_idx = np.argmax(accuracy_list)
best_model_name_idx = best_accuracy_idx % len(model_name_list)
best_model_name = model_name_list[best_model_name_idx]
best_data_word_limit_name_idx = int((best_accuracy_idx - best_model_name_idx) / len(model_name_list))
best_data_word_limit_name = data_word_limit_name_list[best_data_word_limit_name_idx]

print('best_data_word_limit:', best_data_word_limit_name)
print('best_model:', best_model_name)
print('accuracy:', np.max(accuracy_list))

best_data_word_limit: last 2 word
best_model: ANN
accuracy: 0.768722713534087


# Kaggle Submission

## data preprocess

In [38]:
ANN_model = clone_model(model)
ANN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_list = [linear_svc_model, svc_linear_model, svc_poly_model, svc_rbf_model, ANN_model]

if best_data_word_limit_name_idx == 0:
    if best_model_name_idx == 0:
        x = X
        y = Y
    elif best_model_name_idx == 4:
        x = X
        y = Y_onehot
    elif best_model_name_idx > 0 and best_data_word_limit_name_idx < 4:
        x = X_pca
        y = Y
elif best_data_word_limit_name_idx == 1:
    if best_model_name_idx == 0:
        x = X_last_word
        y = Y_last_word
    elif best_model_name_idx == 4:
        x = X_last_word
        y = Y_onehot_last_word
    elif best_model_name_idx > 0 and best_data_word_limit_name_idx < 4:
        x = X_pca_last_word
        y = Y_last_word
elif best_data_word_limit_name_idx == 2:
    if best_model_name_idx == 0:
        x = X_last_2_word
        y = Y_last_2_word
    elif best_model_name_idx == 4:
        x = X_last_2_word
        y = Y_onehot_last_2_word
    elif best_model_name_idx > 0 and best_data_word_limit_name_idx < 4:
        x = X_pca_last_2_word
        y = Y_last_2_word
        
best_model = model_list[best_model_name_idx]

## retrain

In [39]:
if best_model_name_idx < 4:
    best_model = best_model.fit(x, y)
elif best_model_name_idx == 4:
    train_model = best_model.fit(x=x, y=y, epochs=4, batch_size=1024, verbose=2)

Epoch 1/4
29774/29774 - 1s - loss: 1.7086 - acc: 0.5518
Epoch 2/4
29774/29774 - 1s - loss: 0.7717 - acc: 0.7764
Epoch 3/4
29774/29774 - 1s - loss: 0.5162 - acc: 0.8495
Epoch 4/4
29774/29774 - 1s - loss: 0.3591 - acc: 0.8987


## read test.json

In [40]:
test_dataset = pd.read_json('test.json')
if best_data_word_limit_name_idx > 0:
    word_limit = best_data_word_limit_name_idx
    test_dataset[test_dataset.columns[-1]] = test_dataset[test_dataset.columns[-1]].apply(lambda i : [' '.join(((item.split('(')[0]).split(',')[0]).split()[-word_limit:]) for item in i])

In [41]:
print(test_dataset[test_dataset.columns[0]])

0       10210
1        2310
2       33213
3       16902
4        9056
        ...  
9995     5759
9996    47786
9997    37976
9998    42429
9999    12817
Name: id, Length: 10000, dtype: int64


## preprocess x_test

In [42]:
x_test = test_dataset[test_dataset.columns[-1]]
if best_data_word_limit_name_idx == 0:
    x_test = pd.DataFrame(mlb.transform(x_test), columns=mlb.classes_, index=x_test.index)
elif best_data_word_limit_name_idx == 1:
    x_test = pd.DataFrame(mlb_last_word.transform(x_test), columns=mlb_last_word.classes_, index=x_test.index)
elif best_data_word_limit_name_idx == 2:
    x_test = pd.DataFrame(mlb_last_2_word.transform(x_test), columns=mlb_last_2_word.classes_, index=x_test.index) 
x_test = np.array(x_test)
x_test = x_test.astype(float)

# if best_model_name_idx > 0 and best_model_name_idx < 4:
#     if best_data_word_limit_name_idx == 0:
#         n_com_begin = 500
#     elif best_data_word_limit_name_idx == 1:
#         n_com_begin = 105
#     elif best_data_word_limit_name_idx == 2:
#         n_com_begin = 380
#     x_test, x_test_proper_n_com, x_test_explained_ratio = pca_with_proper_n_com(x_test, n_com_begin) 
#     print('n_com:', x_test_proper_n_com, 'explained variance ratio:', x_test_explained_ratio)

if best_model_name_idx > 0 and best_model_name_idx < 4:
    if best_data_word_limit_name_idx == 0:
        n_com = X_proper_n_com
    elif best_data_word_limit_name_idx == 1:
        n_com = X_proper_n_com_last_word
    elif best_data_word_limit_name_idx == 2:
        n_com = X_proper_n_com_last_2_word
    
    pca = PCA(n_components = n_com)
    x_test = pca.fit_transform(x_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['8 juice', 'Artichoke Hearts', 'Baking Blend', 'Bay Leaves', 'Blackened Seasoning', 'Bordelaise sauce', 'Bouillon Powder', 'Budweiser', 'Castelvetrano olives', 'Chambord Liqueur', 'Chartreuse Liqueur', 'Comfort Liqueur', 'Confectioners Sugar', 'Cornbread Mix', 'Creole Seasoning', 'Crushed Tomatoes', 'Elmlea single', 'Fisher Pecans', 'Fume Blanc', 'Garam Masala', 'Garden Parsley', 'Gember Knoflook', 'Gold Tequila', 'Greek Seasoning', 'Greek dressing', 'Heinz Ketchup', 'Ice Cream', 'Italian Sausage', 'Jameson Whiskey', 'Jose Cuervo', 'Kahlua Liqueur', 'Kidney Beans', 'Liquid Aminos', 'Mo Qua', 'Pecan Halves', 'Plus Pasta', 'Sangiovese', 'Sauvignon Blanc', 'Seven Lettuces', 'Spicy Salt', 'Strawberry Kiwi', 'Tea Mix', 'Tomato Ketchup', 'Wok Olie', 'abbamele', 'acai juice', 'aleppo', 'aloe juice', 'and annatto', 'apple rings', 'arctic char', 'arhar', 'arrow root', 'assam', 'awase 

## get pred_y

In [43]:
if best_model_name_idx < 4:
    pred_y = best_model.predict(x_test)
elif best_model_name_idx == 4:
    pred_y = best_model.predict_classes(x_test)
    if best_data_word_limit_name_idx == 0:
        pred_y = Y_le_model.inverse_transform(pred_y)
    elif best_data_word_limit_name_idx == 1:
        pred_y = Y_le_model_last_word.inverse_transform(pred_y)
    elif best_data_word_limit_name_idx == 2:
        pred_y = Y_le_model_last_2_word.inverse_transform(pred_y)

In [44]:
test_dataset = test_dataset.drop(columns=test_dataset.columns[-1])

In [45]:
df_1 = pd.DataFrame(list(test_dataset[test_dataset.columns[0]]), columns=['Id'])
df_2 = pd.DataFrame(pred_y, columns=['Category'])
y_test = pd.concat([df_1, df_2], axis=1)

In [46]:
print(y_test)

         Id      Category
0     10210       italian
1      2310  cajun_creole
2     33213       mexican
3     16902        indian
4      9056       mexican
...     ...           ...
9995   5759       italian
9996  47786    vietnamese
9997  37976      japanese
9998  42429        indian
9999  12817       italian

[10000 rows x 2 columns]


## write prediction to csv

In [47]:
y_test.to_csv('/home/maomao/y_test.csv', index=False)